# Logistic (Shopee Code League 2020)

This scores 1.0 with late submission.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
%matplotlib inline

# Data Exploration

In [2]:
data_path = "./data/delivery_orders_march.csv"
result_path = "./data/result.csv"

In [3]:
%%time
full_data = pd.read_csv(data_path)

CPU times: user 8.47 s, sys: 868 ms, total: 9.34 s
Wall time: 11.9 s


In [4]:
full_data.dtypes

orderid                  int64
pick                     int64
1st_deliver_attempt    float64
2nd_deliver_attempt    float64
buyeraddress            object
selleraddress           object
dtype: object

In [5]:
full_data

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,1585821728,1.585978e+09,NaN,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,1585895011,1.586179e+09,1.586232e+09,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,1585826951,1.585965e+09,NaN,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,1585891194,1.586230e+09,NaN,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


In [6]:
full_data.shape

(3176313, 6)

In [7]:
full_data.orderid.nunique()

3176313

# Implement Solution

Get apart for testing or get all to processing

In [8]:
# For final solution, we will assign `d` to `full_data`
d = full_data

In [9]:
d.shape

(3176313, 6)

Set environment

In [10]:
places = [ "metro manila", "luzon", "visayas", "mindanao" ]

# mapping for city index
places_map = { place:i for i, place in enumerate(places)}

# sla_map[seller][buyer]
sla_map = [[3, 5, 7, 7],
           [5, 5, 7, 7],
           [7, 7, 7, 7],
           [7, 7, 7, 7]]

In [11]:
holidays = [ "2020-03-08", "2020-03-25", "2020-03-30", "2020-03-31" ]

In [12]:
# convert address to city index
def get_place_index(address): 
    return np.argmax([address.lower().rfind(i) for i in places])

Convert `buyeraddress` and `selleraddress` to city index

In [13]:
%%time
d["buyeraddress"] = d["buyeraddress"].apply(lambda x: get_place_index(x))

CPU times: user 22.7 s, sys: 482 ms, total: 23.2 s
Wall time: 25.3 s


In [14]:
%%time
d["selleraddress"] = d["selleraddress"].apply(lambda x: get_place_index(x))

CPU times: user 23.7 s, sys: 465 ms, total: 24.1 s
Wall time: 28.4 s


Fix missing value

In [15]:
d['1st_deliver_attempt'] = d['1st_deliver_attempt'].astype(np.int64)
d['2nd_deliver_attempt'] = d['2nd_deliver_attempt'].fillna(0).astype(np.int64)

Count SLA

In [16]:
%%time
d["sla"] = d.apply(lambda x: sla_map[x["selleraddress"]][x["buyeraddress"]], axis=1)

CPU times: user 1min 14s, sys: 920 ms, total: 1min 15s
Wall time: 1min 20s


Convert to timestamp GMT+8

In [17]:
dt_columns = ['pick', '1st_deliver_attempt', '2nd_deliver_attempt']
for dt_col in dt_columns:
    d[dt_col] = (d[dt_col] + 3600*8) // (3600*24)

In [18]:
d

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,sla
0,2215676524,18323,18326,0,0,0,3
1,2219624609,18325,18327,18331,0,0,3
2,2220979489,18325,18327,0,0,0,3
3,2221066352,18326,18328,0,0,0,3
4,2222478803,18325,18327,0,1,0,5
...,...,...,...,...,...,...,...
3176308,31504087640510,18354,18356,0,0,0,3
3176309,31504147352227,18355,18358,18359,0,0,3
3176310,31504462290482,18354,18356,0,0,0,3
3176311,31504851495943,18355,18359,0,1,0,5


Calculate number of first delivery date and number of second delivery date

In [19]:
%%time
t1 = d['pick'].values.astype('datetime64[D]')
t2 = d['1st_deliver_attempt'].values.astype('datetime64[D]')
t3 = d['2nd_deliver_attempt'].values.astype('datetime64[D]')

d['num_days1'] = np.busday_count(t1, t2, weekmask="1111110", holidays=holidays)
d['num_days2'] = np.busday_count(t2, t3, weekmask="1111110", holidays=holidays)

CPU times: user 164 ms, sys: 25.1 ms, total: 189 ms
Wall time: 266 ms


Check if an order is late

In [20]:
d['is_late'] = (d['num_days1'] > d['sla']) | (d['num_days2'] > 3)
d['is_late'] = d['is_late'].astype(int)

Review result

In [21]:
d

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,sla,num_days1,num_days2,is_late
0,2215676524,18323,18326,0,0,0,3,3,-15708,0
1,2219624609,18325,18327,18331,0,0,3,2,3,0
2,2220979489,18325,18327,0,0,0,3,2,-15709,0
3,2221066352,18326,18328,0,0,0,3,2,-15710,0
4,2222478803,18325,18327,0,1,0,5,2,-15709,0
...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,18354,18356,0,0,0,3,2,-15731,0
3176309,31504147352227,18355,18358,18359,0,0,3,2,1,0
3176310,31504462290482,18354,18356,0,0,0,3,2,-15731,0
3176311,31504851495943,18355,18359,0,1,0,5,3,-15733,0


In [22]:
d[d.is_late != 0].head(10)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,sla,num_days1,num_days2,is_late
7,2224695304,18323,18331,0,0,0,3,7,-15712,1
19,2227790841,18323,18331,18339,3,0,7,7,7,1
21,2227816770,18323,18327,18328,0,0,3,4,1,1
22,2227817800,18323,18332,0,2,0,7,8,-15713,1
24,2227831996,18323,18327,18331,0,0,3,4,3,1
27,2227846339,18323,18332,0,2,0,7,8,-15713,1
29,2227849732,18323,18330,0,0,0,3,6,-15711,1
38,2227878093,18323,18327,0,0,0,3,4,-15709,1
52,2227970321,18323,18327,0,0,0,3,4,-15709,1
60,2228040329,18323,18332,0,1,0,5,8,-15713,1


In [23]:
result = d[["orderid", "is_late"]]

In [24]:
len(result[result.is_late != 0])

762422

In [25]:
result.to_csv(result_path, index=False)